In [19]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from flowprintOptimal.sekigo.flowUtils.commons import loadFlows, saveFlows
from flowprintOptimal.sekigo.flowUtils.sampler import FixedLengthSampler
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation
from typing import List
from flowprintOptimal.sekigo.flowUtils.flowDatasets import ActivityDataset
from torch.utils.data import DataLoader
from flowprintOptimal.sekigo.dataAnalysis.vNATDataFramePeocessor import VNATDataFrameProcessor
from flowprintOptimal.sekigo.flowUtils.commons import getValidInvalidStartingPointsForSubFlowStart
from torch.utils.data import DataLoader
import math
import random
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix,  ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
from joblib import delayed, Parallel
from flowprintOptimal.sekigo.flowUtils.commons import getActivityArrayFromFlow

In [2]:
flows = loadFlows(path= "/Users/rushi/Desktop/UNSW/data/VNAT/flowStore/vnatflows1second.json")

In [3]:
flows = VNATDataFrameProcessor.convertLabelsToTopLevel(flows)

In [4]:
split_flows = FixedLengthSampler.sampleAndCutToLength(data= flows,flow_config= flows[0].flow_config,required_length_in_seconds= 30,ratio_of_median_to_sample= .1, min_activity_for_start_point= 30, temporal_train_ratio= 1, sample_wise_train_ratio= .8)

{'streaming': 0.8293369899566394, 'FT': 1, 'chat': 0.054383513372297895, 'control': 0.6200437782272589}
overlapping points = 0


In [5]:
train_flows = split_flows["train_flows"]
test_flows = split_flows["test_flows"]
activity_dataset_train = ActivityDataset(flows= train_flows)
activity_dataset_test = ActivityDataset(flows= test_flows)

In [6]:
flow = train_flows[0]

In [27]:
def maxNormalizeFlow(flow : FlowRepresentation):

    def normalizeArrayByBand(array : np.ndarray):
        # array is of shape (bands,timesteps), we divide timestamps by the max in each slot
        maxes = array.max(axis= -1,keepdims= True)
        array = array/maxes
        return array
    
    
    feature_array = np.concatenate((flow.up_packets,flow.down_packets,flow.up_bytes,flow.down_bytes),axis= 0)
    feature_array = normalizeArrayByBand(feature_array).T
    normalized_packetlengths = getActivityArrayFromFlow(flow= flow)
    feature_array = np.concatenate((feature_array,normalized_packetlengths), axis= 1)
    return feature_array
        

In [11]:
flow.up_packets/flow.up_packets.max(), flow.down_packets/flow.down_packets.max()

(array([[0.00332226, 0.0166113 , 0.        , 0.00664452, 0.33554817,
         0.30564784, 0.        , 0.        , 0.61461794, 0.01827243,
         0.00332226, 0.00332226, 0.        , 0.        , 0.        ,
         0.00996678, 0.        , 1.        , 0.        , 0.00830565,
         0.00664452, 0.00664452, 0.00332226, 0.00664452, 0.        ,
         0.00166113, 0.        , 0.00166113, 0.27740864, 0.        ],
        [0.        , 0.00166113, 0.        , 0.        , 0.00996678,
         0.        , 0.        , 0.        , 0.00332226, 0.00166113,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.00498339, 0.        , 0.00332226, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.00166113, 0.        ]]),
 array([[1.77619893e-03, 1.06571936e-02, 0.00000000e+00, 3.55239787e-03,
         7.54884547e-02, 7.28241563e-02, 0.00000000e+00, 0.00000000e+00,
         7.46003552e-0